# Authentications
First, let's authenticate the colab notebook to access both GCP and GEE.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

# Imports
Note that we downgraded tensorflow to `2.1.0`. That's because that's the tensorflow runtime version  that we use to train, eeify, and deploy the model later on. So we want to make sure that this colab notebook is using the same version.


In [ ]:
!pip install tensorflow==2.1.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
import folium

# Training Job Preparation
In this section, we'll create a training task with configurations and model definition file.
 

In [ ]:
PACKAGE_PATH = 'cdl_ai_platform'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

In [ ]:
%%writefile {PACKAGE_PATH}/config.py

import tensorflow as tf

# INSERT YOUR PROJECT HERE!
PROJECT = 'GCP project Id'

# INSERT YOUR BUCKET HERE!
BUCKET = 'your bucket'

# Specify names of output locations in Cloud Storage.
JOB_FOLDER = 'your folder'
JOB_DIR = 'gs://' + BUCKET + '/' + JOB_FOLDER + '/trainer'
MODEL_DIR = JOB_DIR + '/model'
LOGS_DIR = JOB_DIR + '/logs'

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = JOB_DIR + '/eeified'

# Pre-computed training and eval data.
DATA_BUCKET = 'bucket from data prep notebook'
FOLDER = 'FOLDER from data prep notebook'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']
BANDS = opticalBands + thermalBands
RESPONSE = 'cultivated'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 50
BUFFER_SIZE = 3000
OPTIMIZER = 'adam'
LOSS = 'binary_crossentropy'
METRICS = ['binary_accuracy']

Let's check if the config file has been written correctly.

In [ ]:
!cat {PACKAGE_PATH}/config.py

from cdl_ai_platform import config
print('\n\n', config.BATCH_SIZE)

Now let's write a model file which first creates a training and evaluation datasets from the data prepared in `DataPrep.ipynb` and then defines the model that the training job needs to train.


In [ ]:
%%writefile {PACKAGE_PATH}/model.py

from . import config
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import metrics
from tensorflow.python.keras import models
from tensorflow.python.keras import optimizers

# Dataset loading functions

def parse_tfrecord(example_proto):
  return tf.io.parse_single_example(example_proto, config.FEATURES_DICT)

def to_tuple(inputs):
  inputsList = [inputs.get(key) for key in config.FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(config.BANDS)], stacked[:,:,len(config.BANDS):]

def get_dataset(pattern):
	glob = tf.io.gfile.glob(pattern)
	dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
	dataset = dataset.map(parse_tfrecord)
	dataset = dataset.map(to_tuple)
	return dataset

def get_training_dataset():
	glob = 'gs://' + config.DATA_BUCKET + '/' + config.FOLDER + '/' + config.TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(config.BUFFER_SIZE).batch(config.BATCH_SIZE).repeat()
	return dataset

def get_eval_dataset():
  glob = 'gs://' + config.DATA_BUCKET + '/' + config.FOLDER + '/' + config.EVAL_BASE + '*'
  dataset = get_dataset(glob)
  dataset = dataset.batch(1).repeat()
  return dataset

# Model Definition

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(config.BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(config.NCLASS, (1,1), activation='softmax')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(config.OPTIMIZER), 
		loss=losses.get(config.LOSS),
		metrics=[metrics.get(metric) for metric in config.METRICS])

	return model

Now let's do the sanity check on both dataset creation and the model definition.

In [ ]:
from cdl_ai_platform import model

eval = model.get_eval_dataset()
print(iter(eval.take(1)).next())

In [ ]:
model = model.get_model()
print(model.summary())

Now let's define the task that the AI Platform Training job will run

In [ ]:
%%writefile {PACKAGE_PATH}/task.py

from . import config
from . import model
import tensorflow as tf

if __name__ == '__main__':

  training = model.get_training_dataset()
  evaluation = model.get_eval_dataset()

  m = model.get_model()

  m.fit(
      x=training,
      epochs=config.EPOCHS, 
      steps_per_epoch=int(config.TRAIN_SIZE / config.BATCH_SIZE), 
      validation_data=evaluation,
      validation_steps=int(config.EVAL_SIZE),
      callbacks=[tf.keras.callbacks.TensorBoard(config.LOGS_DIR)])

  m.save(config.MODEL_DIR, save_format='tf')

# Training Job Submission

In [ ]:
JOB_NAME = 'unique_job_name'
TRAINER_PACKAGE_PATH = 'cdl_ai_platform'
MAIN_TRAINER_MODULE = 'cdl_ai_platform.task'
REGION = 'select your region'

In [ ]:
!gcloud config set project {config.PROJECT}

The following command will create the job on GCP AI platform and your `PROJECT` will be billed for it.

In [ ]:
!gcloud ai-platform jobs submit training {JOB_NAME} \
    --job-dir {config.JOB_DIR}  \
    --package-path {TRAINER_PACKAGE_PATH} \
    --module-name {MAIN_TRAINER_MODULE} \
    --region {REGION} \
    --project {config.PROJECT} \
    --runtime-version 2.1 \
    --python-version 3.7 \
    --scale-tier basic-gpu

# Training Job Monitoring
The training job can be monitored directly from the GCP AI Platform Jobs Console or you can visualize it in the notebook using ***tensorboard***

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.LOGS_DIR}